In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install transformers
# !pip install pytorch_lightning

In [3]:
import argparse
import json
import os
from functools import partial
import pytorch_lightning as pl
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.nn import functional as F
from torch.nn.modules import CrossEntropyLoss
from torch.utils.data.dataloader import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup, RobertaTokenizer
from transformers import RobertaConfig, RobertaModel, AutoTokenizer, AutoModel
from torchmetrics import Accuracy
from typing import List
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

# Args

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device "{device}"')

Using device "cuda"


In [5]:
class parser():
  def __init__(self):
    self.bert_path = "vinai/phobert-base-v2"
    self.batch_size=8
    self.lr=2e-5
    self.workers=0
    self.weight_decay=0.0
    self.adam_epsilon=1e-9
    self.warmup_steps=0
    self.max_length=128
    self.data_dir="D:\\DUCBUI\\DSC\\self\\data\\converted_data_train_tocken.json"
    self.save_path="D:\\DUCBUI\\DSC\\self\\save_model\\model"
    self.save_topk=1
    self.checkpoint_path="D:\\DUCBUI\\DSC\\self\\save_model\\check_point"
    self.span_topk=2
    self.lamb=1.0
    self.task='snli'
    self.mode='train'
    self.max_epochs=10
    self.accumulate_grad_batches=2
    self.scale_pos_neg = 5

args = parser()

# Prepare data


In [6]:
import random
from torch.utils.data import Dataset
import json
import pandas as pd
class LoadDataFromJson(Dataset, ):
    def __init__(self, datasets):
        self.samples = []
        with open(datasets) as file:
            data = json.load(file)

        for id, item in data.items():
            sentences = item["context"]
            if item["evidence"] != None:
              for sentence in sentences:
                data_sample = {}
                evidence = item['evidence'] in sentence or sentence in item['evidence']
                if evidence:
                  data_sample['evidence_label'] = 1
                else:
                  data_sample['evidence_label'] = 0
                data_sample['id'] = id
                data_sample['claim'] = item['claim']
                data_sample['sentence'] = sentence
                data_sample['evidence'] = item['evidence']

                data_sample["verdict"] = item["verdict"]
                data_sample['domain'] = item['domain']
                self.samples.append(data_sample)

    def to_dataframe(self):
        return pd.DataFrame(self.samples).sample(frac=1, random_state=3)


In [7]:
a = LoadDataFromJson(args.data_dir)
df = a.to_dataframe()


In [8]:
df

,evidence_label,id,claim,sentence,evidence,verdict,domain
234309,0,30927,khi gia_nhập college compass nhung được kết_nố...,trong bài_luận gửi đến trường harvard nữ_sinh_...,khi trở_thành học_sinh của college compass nhu...,SUPPORTED,giaoduc
223745,0,30428,từ lúc nghỉ_việc gia_đình chị thúy rất khó khắ...,lê tuyết,từ lúc nghỉ_việc gia_đình chị thúy chỉ kiếm đư...,SUPPORTED,thoisu
94193,0,11558,cần nhiều công_nghệ và thời_gian để khắc_phục ...,khi tìm_kiếm sản_phẩm chăm_sóc da đặc_biệt cho...,bác_sĩ thành cho rằng trường_hợp bệnh_nhân trê...,SUPPORTED,suckhoe
114060,0,12370,ông trump không_thể tự ân_xá cho mình dù trở_t...,quá_trình xử_lý vụ án có_thể diễn ra trong thờ...,nếu trở_thành tổng_thống ông trump không nắm g...,SUPPORTED,thegioi
193742,0,19637,phòng lao_động quận 7 ban quản_lý các khu chế_...,khảo_sát của công_đoàn cho thấy khoảng 240 lao...,hơn tháng qua công_đoàn đã trực_tiếp liên_hệ v...,SUPPORTED,thoisu
...,...,...,...,...,...,...,...
11261,0,7631,tháng 4/2020 là thời_kỳ đỉnh_điểm nhất của cov...,đặc_biệt sản_phẩm này có tiềm_năng hỗ_trợ ngườ...,giai_đoạn cao_điểm covid19 hồi tháng 4 năm 202...,SUPPORTED,khoahoc
48056,0,9398,vật_liệu điện_sắc được giới khoa_học quan_tâm ...,thiết_bị điện_sắc được nhóm nghiên_cứu thử_ngh...,theo nhóm vật_liệu điện_sắc được giới khoa_học...,SUPPORTED,khoahoc
77049,0,10830,những chiến_thắng được ca_ngợi giúp prigozhin ...,các thành_viên wagner được cho là đã âm_thầm t...,điều đó cũng giúp prigozhin tăng_cường ảnh_hưở...,SUPPORTED,thegioi
198296,0,19911,những bức ảnh được nhiếp_ảnh gia_john thomson ...,áo_dài năm thân_may theo phong_cách kín_đáo đư...,thiếu_nữ sài_gòn là một trong những bức ảnh đư...,SUPPORTED,thegioi


In [9]:
df["evidence_label"].value_counts()

evidence_label
0    322111
1     14550
Name: count, dtype: int64

In [10]:
filtered_df = df.loc[(df['sentence'].apply(lambda x: len(x.split())) < 4)]
filtered_df

,evidence_label,id,claim,sentence,evidence,verdict,domain
223745,0,30428,từ lúc nghỉ_việc gia_đình chị thúy rất khó khắ...,lê tuyết,từ lúc nghỉ_việc gia_đình chị thúy chỉ kiếm đư...,SUPPORTED,thoisu
54650,0,9735,giảo cổ_lam,thanh hoa,giảo cổ_lam trong giảo cổ_lam có chứa phanosid,SUPPORTED,suckhoe
162702,0,14185,đầu năm 2022 dự_án sẽ đi vào hoạt_động,để phòng nguy_hiểm,đầu năm 2022 công_trình đi vào hoạt_động,SUPPORTED,thoisu
160332,0,14075,rạng sáng nay người_dân khu_vực gần cầu kênh n...,ngọc tài nguyễn_khánh,rạng sáng nay người_dân gần cầu bắc qua kênh n...,SUPPORTED,thoisu
214061,0,30071,trong năm đầu_tiên cô đã hoàn_thành_phần nghiê...,thanh hằng,năm đầu_tiên cô hoàn_thành_phần nghiên_cứu lý_...,SUPPORTED,giaoduc
...,...,...,...,...,...,...,...
227131,0,30588,trong thông_báo vừa gửi 7 hãng taxi về giá dịc...,gia_minh,trong thông_báo vừa gửi 7 hãng taxi về giá dịc...,SUPPORTED,thoisu
198769,0,19936,yoga xuất_hiện từ thời ấn_độ cổ_đại,bảo đức,yoga là bộ_môn truyền_thống năm nghìn năm_tuổi...,SUPPORTED,thegioi
185884,1,19243,dự_kiến tổng mức đầu_tư cho dự_án cảng_hàng_kh...,phước tuấn_gia chính,phước tuấn_gia chính,SUPPORTED,thoisu
306027,0,46364,ông nguyễn_quang_thái thành_viên hội_đồng tư_v...,hà_an,ông nguyễn_quang_thái thành_viên hội_đồng tư_v...,REFUTED,khoahoc


In [11]:
df = df.drop(df[df.isin(filtered_df.to_dict('list')).all(1)].index)


In [12]:
df

,evidence_label,id,claim,sentence,evidence,verdict,domain
234309,0,30927,khi gia_nhập college compass nhung được kết_nố...,trong bài_luận gửi đến trường harvard nữ_sinh_...,khi trở_thành học_sinh của college compass nhu...,SUPPORTED,giaoduc
94193,0,11558,cần nhiều công_nghệ và thời_gian để khắc_phục ...,khi tìm_kiếm sản_phẩm chăm_sóc da đặc_biệt cho...,bác_sĩ thành cho rằng trường_hợp bệnh_nhân trê...,SUPPORTED,suckhoe
114060,0,12370,ông trump không_thể tự ân_xá cho mình dù trở_t...,quá_trình xử_lý vụ án có_thể diễn ra trong thờ...,nếu trở_thành tổng_thống ông trump không nắm g...,SUPPORTED,thegioi
193742,0,19637,phòng lao_động quận 7 ban quản_lý các khu chế_...,khảo_sát của công_đoàn cho thấy khoảng 240 lao...,hơn tháng qua công_đoàn đã trực_tiếp liên_hệ v...,SUPPORTED,thoisu
349,0,7145,số_liệu của viện phát_triển giáo_dục hàn_quốc ...,tôi nghĩ không nhiều đứa trẻ gây ra nỗi đau_th...,lee cũng cho rằng nhiều học_sinh đã mất niềm t...,SUPPORTED,giaoduc
...,...,...,...,...,...,...,...
11261,0,7631,tháng 4/2020 là thời_kỳ đỉnh_điểm nhất của cov...,đặc_biệt sản_phẩm này có tiềm_năng hỗ_trợ ngườ...,giai_đoạn cao_điểm covid19 hồi tháng 4 năm 202...,SUPPORTED,khoahoc
48056,0,9398,vật_liệu điện_sắc được giới khoa_học quan_tâm ...,thiết_bị điện_sắc được nhóm nghiên_cứu thử_ngh...,theo nhóm vật_liệu điện_sắc được giới khoa_học...,SUPPORTED,khoahoc
77049,0,10830,những chiến_thắng được ca_ngợi giúp prigozhin ...,các thành_viên wagner được cho là đã âm_thầm t...,điều đó cũng giúp prigozhin tăng_cường ảnh_hưở...,SUPPORTED,thegioi
198296,0,19911,những bức ảnh được nhiếp_ảnh gia_john thomson ...,áo_dài năm thân_may theo phong_cách kín_đáo đư...,thiếu_nữ sài_gòn là một trong những bức ảnh đư...,SUPPORTED,thegioi


In [13]:
df_eval_1 = df[df['evidence_label'] == 1].head(690)
df_eval_0 = df[df['evidence_label'] == 0].head(690)
df_eval = pd.concat([df_eval_0, df_eval_1])

In [14]:
df_train = df[~df.isin(df_eval).all(axis=1)]

In [15]:
df_train_1 = df_train[df_train['evidence_label'] == 1]
df_train_0 = df_train[df_train['evidence_label'] == 0].head(len(df_train_1)*args.scale_pos_neg)
df_train = pd.concat([df_train_0, df_train_1])

In [16]:
df_train['evidence_label'].value_counts()

evidence_label
0    66280
1    13256
Name: count, dtype: int64

In [17]:
df_train.duplicated().sum()

0

In [18]:
# debug
df_eval = df_eval.head(10)
df_train = df_train.head(100)

# Model

## Colate function

In [19]:
def collate_to_max_length(batch: List[List[torch.Tensor]], max_len: int = None, fill_values: List[float] = None) -> \
    List[torch.Tensor]:
    """
    pad to maximum length of this batch
    Args:
        batch: a batch of samples, each contains a list of field data(Tensor), which shape is [seq_length]
        max_len: specify max length
        fill_values: specify filled values of each field
    Returns:
        output: list of field batched data, which shape is [batch, max_length]
    """
    # [batch, num_fields]
    lengths = np.array([[len(field_data) for field_data in sample] for sample in batch])
    batch_size, num_fields = lengths.shape
    fill_values = fill_values or [0.0] * num_fields
    # [num_fields]
    max_lengths = lengths.max(axis=0)
    if max_len:
        assert max_lengths.max() <= max_len
        max_lengths = np.ones_like(max_lengths) * max_len

    output = [torch.full([batch_size, max_lengths[field_idx]],
                         fill_value=fill_values[field_idx],
                         dtype=batch[0][field_idx].dtype)
              for field_idx in range(num_fields)]
    for sample_idx in range(batch_size):
        for field_idx in range(num_fields):
            # seq_length
            data = batch[sample_idx][field_idx]
            output[field_idx][sample_idx][: data.shape[0]] = data
    # generate span_index and span_mask
    max_sentence_length = max_lengths[0]
    start_indexs = []
    end_indexs = []
    for i in range(1, max_sentence_length - 1):
        for j in range(i, max_sentence_length - 1):
            # # span大小为10
            # if j - i > 10:
            #     continue
            start_indexs.append(i)
            end_indexs.append(j)
    # generate span mask
    span_masks = []
    for input_ids, label, length in batch:
        span_mask = []
        middle_index = input_ids.tolist().index(2)
        for start_index, end_index in zip(start_indexs, end_indexs):
            if 1 <= start_index <= length.item() - 2 and 1 <= end_index <= length.item() - 2 and (
                start_index > middle_index or end_index < middle_index):
                span_mask.append(0)
            else:
                span_mask.append(1e6)
        span_masks.append(span_mask)
    # add to output
    output.append(torch.LongTensor(start_indexs))
    output.append(torch.LongTensor(end_indexs))
    output.append(torch.LongTensor(span_masks))
    return output  # (input_ids, labels, length, start_indexs, end_indexs, span_masks)



## Data loader

In [20]:
import json
import os
from functools import partial

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer

class DSC_Dataset(Dataset):
    def __init__(self, dataframe, bert_path, max_length: int = 512):
        super().__init__()
        self.max_length = max_length
        # label_map = {"NON": 0, 'SR': 1}
        # with open(os.path.join(directory, prefix + '.json')) as file:
        #     data = json.load(file)
        # self.result = data
        self.result = dataframe
        self.tokenizer = AutoTokenizer.from_pretrained(bert_path, use_fast=False)
        # self.tokenizer = RobertaTokenizer.from_pretrained(bert_path)

    def __len__(self):
        return len(self.result)

    def __getitem__(self, idx):
        # sentence_1, sentence_2, label = self.result[idx]
        # check for required keys
        # if not all(key in self.result[idx] for key in ["claim", "sentence", "verdict"]):
        #     raise KeyError("Missing key in result dictionary.")

        sentence_1 = self.result.iloc[idx]["claim"]
        sentence_2 = self.result.iloc[idx]["sentence"]
        label = self.result.iloc[idx]["evidence_label"]
        # remove .
        if sentence_1.endswith("."):
            sentence_1 = sentence_1[:-1]
        if sentence_2.endswith("."):
            sentence_2 = sentence_2[:-1]
        sentence_1_input_ids = self.tokenizer.encode(
            sentence_1, add_special_tokens=False)
        sentence_2_input_ids = self.tokenizer.encode(
            sentence_2, add_special_tokens=False)
        input_ids = sentence_1_input_ids + [2] + sentence_2_input_ids
        if len(input_ids) > self.max_length - 2:
            input_ids = input_ids[:self.max_length - 2]
        # convert list to tensor
        length = torch.LongTensor([len(input_ids) + 2])
        input_ids = torch.LongTensor([0] + input_ids + [2])
        label = torch.LongTensor([label])
        return input_ids, label, length


# def unit_test():
#     root_path = "D:\\DUCBUI\\DSC\\self_explanining\\data\\"
#     bert_path = "roberta-base"
#     prefix = "train_evidence"
#     dataset = DSC_Dataset(directory=root_path,
#                           prefix=prefix, bert_path=bert_path)

#     print(dataset)



# if __name__ == '__main__':
#     unit_test()

In [21]:
class ExplainableModel(nn.Module):
    def __init__(self, bert_dir):
        super().__init__()
        self.bert_config = RobertaConfig.from_pretrained(
            bert_dir, output_hidden_states=True)
        self.intermediate = RobertaModel.from_pretrained(bert_dir)
        self.span_info_collect = SICModel(self.bert_config.hidden_size)
        self.interpretation = InterpretationModel(self.bert_config.hidden_size)
        self.output = nn.Linear(
            self.bert_config.hidden_size, self.bert_config.num_labels)

    def forward(self, input_ids, start_indexs, end_indexs, span_masks):
        # generate mask
        attention_mask = (input_ids != 1).long()
        # intermediate layer
        a = self.intermediate(
            input_ids, attention_mask=attention_mask)
        hidden_states = self.intermediate(
            input_ids, attention_mask=attention_mask).last_hidden_state
        # span info collecting layer(SIC)
        h_ij = self.span_info_collect(hidden_states, start_indexs, end_indexs)
        # interpretation layer
        H, a_ij = self.interpretation(h_ij, span_masks)
        # output layer
        out = self.output(H)
        return out, a_ij


class SICModel(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size

        self.W_1 = nn.Linear(hidden_size, hidden_size)
        self.W_2 = nn.Linear(hidden_size, hidden_size)
        self.W_3 = nn.Linear(hidden_size, hidden_size)
        self.W_4 = nn.Linear(hidden_size, hidden_size)

    def forward(self, hidden_states, start_indexs, end_indexs):
        W1_h = self.W_1(hidden_states)  # (bs, length, hidden_size)
        W2_h = self.W_2(hidden_states)
        W3_h = self.W_3(hidden_states)
        W4_h = self.W_4(hidden_states)

        # (bs, span_num, hidden_size)
        W1_hi_emb = torch.index_select(W1_h, 1, start_indexs)
        W2_hj_emb = torch.index_select(W2_h, 1, end_indexs)
        W3_hi_start_emb = torch.index_select(W3_h, 1, start_indexs)
        W3_hi_end_emb = torch.index_select(W3_h, 1, end_indexs)
        W4_hj_start_emb = torch.index_select(W4_h, 1, start_indexs)
        W4_hj_end_emb = torch.index_select(W4_h, 1, end_indexs)

        # [w1*hi, w2*hj, w3(hi-hj), w4(hi⊗hj)]
        span = W1_hi_emb + W2_hj_emb + \
            (W3_hi_start_emb - W3_hi_end_emb) + \
            torch.mul(W4_hj_start_emb, W4_hj_end_emb)
        h_ij = torch.tanh(span)
        return h_ij


class InterpretationModel(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.h_t = nn.Linear(hidden_size, 1)

    def forward(self, h_ij, span_masks):
        o_ij = self.h_t(h_ij).squeeze(-1)  # (ba, span_num)
        # mask illegal span
        o_ij = o_ij - span_masks
        # normalize all a_ij, a_ij sum = 1
        a_ij = nn.functional.softmax(o_ij, dim=1)
        # weight average span representation to get H
        H = (a_ij.unsqueeze(-1) * h_ij).sum(dim=1)  # (bs, hidden_size)
        return H, a_ij


# def main():
#     # data
#     input_id_1 = torch.LongTensor([0, 4, 5, 6, 7, 2])
#     input_id_2 = torch.LongTensor([0, 4, 5, 2])
#     input_id_3 = torch.LongTensor([0, 4, 2])
#     batch = [(input_id_1, torch.LongTensor([1]), torch.LongTensor([6])),
#              (input_id_2, torch.LongTensor([1]), torch.LongTensor([4])),
#              (input_id_3, torch.LongTensor([1]), torch.LongTensor([3]))]

#     output = collate_to_max_length(batch=batch, fill_values=[1, 0, 0])
#     input_ids, labels, length, start_indexs, end_indexs, span_masks = output

#     # model
#     bert_path = "roberta-base"
#     model = ExplainableModel(bert_path)
#     print(model)

#     output = model(input_ids, start_indexs, end_indexs, span_masks)
#     print(output[0])


# if __name__ == '__main__':
#     main()


In [22]:
model = ExplainableModel(args.bert_path)

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
model

ExplainableModel(
  (intermediate): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm

In [24]:
# Xuất các layer của mô hình
for name, module in model.named_modules():
    print(name, end="*******\n")
    print(module)
    # print(name)

*******
ExplainableModel(
  (intermediate): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (L

In [25]:
# Xuất các layer của mô hình
for name, module in model.named_modules():

    print(name)


intermediate
intermediate.embeddings
intermediate.embeddings.word_embeddings
intermediate.embeddings.position_embeddings
intermediate.embeddings.token_type_embeddings
intermediate.embeddings.LayerNorm
intermediate.embeddings.dropout
intermediate.encoder
intermediate.encoder.layer
intermediate.encoder.layer.0
intermediate.encoder.layer.0.attention
intermediate.encoder.layer.0.attention.self
intermediate.encoder.layer.0.attention.self.query
intermediate.encoder.layer.0.attention.self.key
intermediate.encoder.layer.0.attention.self.value
intermediate.encoder.layer.0.attention.self.dropout
intermediate.encoder.layer.0.attention.output
intermediate.encoder.layer.0.attention.output.dense
intermediate.encoder.layer.0.attention.output.LayerNorm
intermediate.encoder.layer.0.attention.output.dropout
intermediate.encoder.layer.0.intermediate
intermediate.encoder.layer.0.intermediate.dense
intermediate.encoder.layer.0.intermediate.intermediate_act_fn
intermediate.encoder.layer.0.output
intermedia

In [26]:
# Đếm số lượng tham số huấn luyện được
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Số lượng tham số huấn luyện của mô hình Transformer:", trainable_params)

Số lượng tham số huấn luyện của mô hình Transformer: 137362947


In [27]:

# Đóng băng các lớp trước layer11.output
layer11_found = False
for name, param in model.named_parameters():
    if name.startswith("intermediate.encoder.layer.11.output"):
        layer11_found = True
    if not layer11_found:
        param.requires_grad_(False)

In [28]:
# Đếm số lượng tham số huấn luyện được
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Số lượng tham số huấn luyện của mô hình Transformer:", trainable_params)

Số lượng tham số huấn luyện của mô hình Transformer: 5316867


In [29]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [30]:
phobert

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

# Trainer

In [31]:
import argparse
import json
import os
from functools import partial

import pytorch_lightning as pl
import torch

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.nn import functional as F
from torch.nn.modules import CrossEntropyLoss
from torch.utils.data.dataloader import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup, RobertaTokenizer
from torchmetrics import Accuracy

import os
import sys

import random
random.seed(0)

class ExplainNLP(pl.LightningModule):

    def __init__(
        self,
        args: argparse.Namespace
    ):
        """Initialize a model, tokenizer and config."""
        super().__init__()
        self.args = args
        if isinstance(args, argparse.Namespace):
            self.save_hyperparameters(args)
        self.model = ExplainableModel(args.bert_path)
        # self.tokenizer = RobertaTokenizer.from_pretrained(args.bert_path)
        self.tokenizer = AutoTokenizer.from_pretrained(args.bert_path)
        self.loss_fn = CrossEntropyLoss(weight=torch.tensor([args.scale_pos_neg,1.]).to(device))
        self.train_acc = Accuracy(task="binary")
        self.valid_acc = Accuracy(task="binary")
        self.acc = 0
        self.loss = 0
        self.output = []
        self.check_data = []

    def configure_optimizers(self):
        """Prepare optimizer and schedule (linear warmup and decay)"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.args.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          betas=(0.9, 0.98),  # according to RoBERTa paper
                          lr=self.args.lr,
                          eps=self.args.adam_epsilon)
        t_total = len(self.train_dataloader()) // self.args.accumulate_grad_batches * self.args.max_epochs
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=self.args.warmup_steps,
                                                    num_training_steps=t_total)

        return [optimizer], [{"scheduler": scheduler, "interval": "step"}]

    def forward(self, input_ids, start_indexs, end_indexs, span_masks):
        return self.model(input_ids, start_indexs, end_indexs, span_masks)

    def compute_loss_and_acc(self, batch, mode='train'):
        input_ids, labels, length, start_indexs, end_indexs, span_masks = batch
        y = labels.view(-1)
        y_hat, a_ij = self.forward(input_ids, start_indexs, end_indexs, span_masks)
        # compute loss
        ce_loss = self.loss_fn(y_hat, y)
        reg_loss = self.args.lamb * a_ij.pow(2).sum(dim=1).mean()
        loss = ce_loss - reg_loss
        # compute acc
        predict_scores = F.softmax(y_hat, dim=1)
        predict_labels = torch.argmax(predict_scores, dim=-1)
        if mode == 'train':
            acc = self.train_acc(predict_labels, y)
        else:
            acc = self.valid_acc(predict_labels, y)
        # if test, save extract spans
        if mode == 'test':
            values, indices = torch.topk(a_ij, self.args.span_topk)
            values = values.tolist()
            indices = indices.tolist()
            for i in range(len(values)):
                input_ids_list = input_ids[i].tolist()
                origin_sentence = self.tokenizer.decode(input_ids_list, skip_special_tokens=True)
                self.output.append(
                    str(labels[i].item()) + '<->' + str(predict_labels[i].item()) + '<->' + origin_sentence + '\n')
                # print()
                for j, span_idx in enumerate(indices[i]):
                    score = values[i][j]
                    start_index = start_indexs[span_idx]
                    end_index = end_indexs[span_idx]
                    pre = self.tokenizer.decode(input_ids_list[:start_index], skip_special_tokens=True)
                    high_light = self.tokenizer.decode(input_ids_list[start_index:end_index + 1],
                                                       skip_special_tokens=True)
                    post = self.tokenizer.decode(input_ids_list[end_index + 1:], skip_special_tokens=True)
                    span_sentence = pre + '【' + high_light + '】' + post
                    self.output.append(format('%.4f' % score) + "->" + span_sentence + '\n')
                    # print(format('%.4f' % score), "->", span_sentence)
                    if j == 0:
                        # generate data for check progress
                        self.check_data.append(str(labels[i].item()) + '\t' + high_light + '\n')
                self.output.append('\n')
            # print('='*30)

        return loss, acc

    def on_validation_epoch_end(self):
        # log epoch metric
        self.valid_acc.compute()
        self.log('valid_acc_end', self.valid_acc.compute())

    def train_dataloader(self) -> DataLoader:
        return self.get_dataloader("train")

    def training_step(self, batch, batch_idx):
        loss, acc = self.compute_loss_and_acc(batch)
        self.loss = loss
        self.acc = acc
        self.log('lr', self.trainer.optimizers[0].param_groups[0]['lr'])
        self.log('train_acc', acc, on_step=True, on_epoch=False)
        self.log('train_loss', loss)
        return loss

    def val_dataloader(self):
        return self.get_dataloader("dev")

    def validation_step(self, batch, batch_idx):
        loss, acc = self.compute_loss_and_acc(batch, mode='dev')
        self.loss = loss
        self.acc = acc
        self.log('valid_acc', acc, on_step=False, on_epoch=True)
        self.log('valid_loss', loss)
        return loss

    def get_dataloader(self, prefix="train") -> DataLoader:
        """get training dataloader"""
        if prefix == "train":
          dataset = DSC_Dataset(df_train,
                                bert_path=args.bert_path,
                                max_length=self.args.max_length)
        else:
          dataset = DSC_Dataset(df_eval,
                                bert_path=args.bert_path,
                                max_length=self.args.max_length)
        dataloader = DataLoader(
            dataset=dataset,
            batch_size=self.args.batch_size,
            num_workers=self.args.workers,
            collate_fn=partial(collate_to_max_length, fill_values=[1, 0, 0]),
            drop_last=False
        )
        return dataloader

    def test_dataloader(self):
        return self.get_dataloader("test")

    def test_step(self, batch, batch_idx):
        loss, acc = self.compute_loss_and_acc(batch, mode='test')
        self.loss = loss
        self.acc = acc
        return {'test_loss': loss, "test_acc": acc}

    def test_epoch_end(self, outputs):
        with open(os.path.join(self.args.save_path, 'output.txt'), 'w', encoding='utf8') as f:
            f.writelines(self.output)
        with open(os.path.join(self.args.save_path, 'test.txt'), 'w', encoding='utf8') as f:
            f.writelines(self.check_data)
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss, 'test_acc': avg_acc}
        print(avg_loss, avg_acc)
        return {'val_loss': avg_loss, 'log': tensorboard_logs}





def train(args):
    # if save path does not exits, create it
    if not os.path.exists(args.save_path):
        os.mkdir(args.save_path)

    model = ExplainNLP(args)

    # # freeze layers before layer.11.output.
    # layer11_found = False
    # for name, param in model.named_parameters():
    #     if name.startswith("intermediate.encoder.layer.11.output"):
    #         layer11_found = True
    #     if not layer11_found:
    #         param.requires_grad_(False)

    a = 0
    def get_a(a):
        a += 1
        return a
    # checkpoint_callback = ModelCheckpoint(
    #     dirpath=os.path.join(args.save_path, f'{get_a(a)}'),
    #     save_top_k=args.save_topk,
    #     save_last=True,
    #     monitor="valid_acc_end",
    #     mode="max",
    # )
    checkpoint_callback = ModelCheckpoint(
        dirpath=args.save_path,
        save_top_k=args.save_topk,
        save_last=True,
        monitor="valid_acc_end",
        mode="max",
        filename='{epoch}-{model.acc:.2f}-{model.loss:.2f}',
        save_weights_only=True,
    )

    logger = TensorBoardLogger(
        save_dir=args.save_path,
        name='log'
    )

    # save args
    with open(os.path.join(args.save_path, "args.json"), 'w') as f:
        args_dict = args.__dict__
        # del args_dict['tpu_cores']
        json.dump(args_dict, f, indent=4)

    # trainer = Trainer(logger=logger)
    trainer = Trainer(callbacks=checkpoint_callback, logger=logger)

    trainer.fit(model)


def evaluate(args):
    model = ExplainNLP(args)
    checkpoint = torch.load(args.checkpoint_path, map_location=torch.device('cuda'))
    model.load_state_dict(checkpoint['state_dict'])
    trainer = Trainer()
    trainer.test(model)


def main():
    # parser = get_parser()
    # # parser = Trainer.add_argparse_args(parser)
    # args = parser.parse_args()
    if args.mode == 'train':
        train(args)
    elif args.mode == 'eval':
        evaluate(args)
    else:
        raise Exception("unexpected mode!!!")


if __name__ == '__main__':
    from multiprocessing import freeze_support

    freeze_support()
    main()

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\buidu\anaconda3\envs\nlp\lib\site-packages\pytorch_lightning\loops\utilities.py:72: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
Missing logger folder: D:\DUCBUI\DSC\self\save_model\model\log
c:\Users\buidu\anaconda3\envs\nlp\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:630: Checkpoint directory D:\DUCBUI\DSC\self\save_model\model exist

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\buidu\anaconda3\envs\nlp\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
C:\Users\buidu\AppData\Local\Temp\ipykernel_19748\3543186699.py:57: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  output.append(torch.LongTensor(span_masks))
c:\Users\buidu\anaconda3\envs\nlp\lib\site-packages\pytorch_lightning\utilities\data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
c:\Users\buidu\anaconda3

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

c:\Users\buidu\anaconda3\envs\nlp\lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
